In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = 'none'
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tqdm import tqdm
from tifffile import imread
from csbdeep.utils import Path, normalize


from stardist.matching import matching, matching_dataset
from stardist.models import Config2D, StarDist2D, StarDistData2D
from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available

np.random.seed(42)
lbl_cmap = random_label_cmap()

In [2]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    print('CUDA is available')
else:
    print('CUDA is not available')


CUDA is available


In [3]:
import numpy as np
import xml.etree.ElementTree as ET
import cv2
import random
import matplotlib.pyplot as plt
from skimage import io, transform

def he_to_binary_mask_final(filename):
    im_file = filename + '.tif'
    xml_file = filename + '.xml'

    # Parse the XML file
    xDoc = ET.parse(xml_file)
    Regions = xDoc.findall('.//Region')  # get a list of all the region tags
    xy = []

    for region in Regions:
        vertices = region.findall('.//Vertex')
        xy_region = np.zeros((len(vertices), 2))

        for i, vertex in enumerate(vertices):
            x = float(vertex.get('X'))
            y = float(vertex.get('Y'))
            xy_region[i, :] = [x, y]

        xy.append(xy_region)

    im_info = cv2.imread(im_file)

    nrow, ncol, _ = im_info.shape
    binary_mask = np.zeros((nrow, ncol), dtype=np.uint8)
    color_mask = np.zeros((nrow, ncol, 3), dtype=np.float32)

    for zz, vertices in enumerate(xy, start=1):
        #print(f'Processing object #{zz}')
        smaller_x = vertices[:, 0].astype(int)
        smaller_y = vertices[:, 1].astype(int)

        # Create a mask and add it to the current mask
        # Use logical OR to handle overlapping objects
        polygon = cv2.fillPoly(np.zeros_like(binary_mask), [vertices.astype(int)], 1)
        binary_mask = binary_mask + zz * (1 - np.minimum(1, binary_mask)) * polygon

        # Create a random color mask
        color_mask += np.stack([random.random() * polygon for _ in range(3)], axis=2)

    
    return binary_mask, im_info



X_test=[]
Y_test=[]
for url in tqdm(glob(r'C:\Users\DELL\Desktop\Final Work StarDist\StarDist IIT\data/MoNuSegTestData/*.tif')):
    # # Call the function with your filename
    binary_mask, image = he_to_binary_mask_final(url.split(".")[0])
    height, width, _ = image.shape
    image = transform.resize(image, (height, width))
    X_test.append(image)
    Y_test.append(binary_mask)

100%|██████████| 14/14 [02:48<00:00, 12.02s/it]


In [4]:
def plot_img_label(img, lbl, img_title="image", lbl_title="label", **kwargs):
    fig, (ai,al) = plt.subplots(1,2, figsize=(12,5), gridspec_kw=dict(width_ratios=(1.25,1)))
    im = ai.imshow(img, cmap='gray', clim=(0,1))
    ai.set_title(img_title)    
    fig.colorbar(im, ax=ai)
    al.imshow(lbl, cmap=lbl_cmap)
    al.set_title(lbl_title)
    plt.tight_layout()

In [5]:
model = StarDist2D(None, name='densenet_segnet_iit', basedir='models')

Model: "densenet169"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   896         ['input[0][0]']                  
                                32)                                                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, None, None,   9248        ['conv2d[0][0]']                 
                                32)                                                     

                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, None, None,   512        ['conv2_block3_1_conv[0][0]']    
 ization)                       128)                                                              
                                                                                                  
 conv2_block3_1_relu (Activatio  (None, None, None,   0          ['conv2_block3_1_bn[0][0]']      
 n)                             128)                                                              
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, None, None,   36864       ['conv2_block3_1_relu[0][0]']    
                                32)                                                               
                                                                                                  
 conv2_blo

                                128)                                                              
                                                                                                  
 pool2_pool (AveragePooling2D)  (None, None, None,   0           ['pool2_conv[0][0]']             
                                128)                                                              
                                                                                                  
 conv3_block1_0_bn (BatchNormal  (None, None, None,   512        ['pool2_pool[0][0]']             
 ization)                       128)                                                              
                                                                                                  
 conv3_block1_0_relu (Activatio  (None, None, None,   0          ['conv3_block1_0_bn[0][0]']      
 n)                             128)                                                              
          

 conv3_block4_2_conv (Conv2D)   (None, None, None,   36864       ['conv3_block4_1_relu[0][0]']    
                                32)                                                               
                                                                                                  
 conv3_block4_concat (Concatena  (None, None, None,   0          ['conv3_block3_concat[0][0]',    
 te)                            256)                              'conv3_block4_2_conv[0][0]']    
                                                                                                  
 conv3_block5_0_bn (BatchNormal  (None, None, None,   1024       ['conv3_block4_concat[0][0]']    
 ization)                       256)                                                              
                                                                                                  
 conv3_block5_0_relu (Activatio  (None, None, None,   0          ['conv3_block5_0_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block8_2_conv (Conv2D)   (None, None, None,   36864       ['conv3_block8_1_relu[0][0]']    
                                32)                                                               
                                                                                                  
 conv3_block8_concat (Concatena  (None, None, None,   0          ['conv3_block7_concat[0][0]',    
 te)                            384)                              'conv3_block8_2_conv[0][0]']    
                                                                                                  
 conv3_block9_0_bn (BatchNormal  (None, None, None,   1536       ['conv3_block8_concat[0][0]']    
 ization)                       384)                                                              
                                                                                                  
 conv3_blo

...

 n)                             128)                                                              
                                                                                                  
 conv2_block4_2_conv (Conv2D)   (None, None, None,   36864       ['conv2_block4_1_relu[0][0]']    
                                32)                                                               
                                                                                                  
 conv2_block4_concat (Concatena  (None, None, None,   0          ['conv2_block3_concat[0][0]',    
 te)                            192)                              'conv2_block4_2_conv[0][0]']    
                                                                                                  
 conv2_block5_0_bn (BatchNormal  (None, None, None,   768        ['conv2_block4_concat[0][0]']    
 ization)                       192)                                                              
          

 conv3_block2_0_bn (BatchNormal  (None, None, None,   640        ['conv3_block1_concat[0][0]']    
 ization)                       160)                                                              
                                                                                                  
 conv3_block2_0_relu (Activatio  (None, None, None,   0          ['conv3_block2_0_bn[0][0]']      
 n)                             160)                                                              
                                                                                                  
 conv3_block2_1_conv (Conv2D)   (None, None, None,   20480       ['conv3_block2_0_relu[0][0]']    
                                128)                                                              
                                                                                                  
 conv3_block2_1_bn (BatchNormal  (None, None, None,   512        ['conv3_block2_1_conv[0][0]']    
 ization) 

                                                                                                  
 conv3_block6_0_bn (BatchNormal  (None, None, None,   1152       ['conv3_block5_concat[0][0]']    
 ization)                       288)                                                              
                                                                                                  
 conv3_block6_0_relu (Activatio  (None, None, None,   0          ['conv3_block6_0_bn[0][0]']      
 n)                             288)                                                              
                                                                                                  
 conv3_block6_1_conv (Conv2D)   (None, None, None,   36864       ['conv3_block6_0_relu[0][0]']    
                                128)                                                              
                                                                                                  
 conv3_blo

 te)                            416)                              'conv3_block9_2_conv[0][0]']    
                                                                                                  
 conv3_block10_0_bn (BatchNorma  (None, None, None,   1664       ['conv3_block9_concat[0][0]']    
 lization)                      416)                                                              
                                                                                                  
 conv3_block10_0_relu (Activati  (None, None, None,   0          ['conv3_block10_0_bn[0][0]']     
 on)                            416)                                                              
                                                                                                  
 conv3_block10_1_conv (Conv2D)  (None, None, None,   53248       ['conv3_block10_0_relu[0][0]']   
                                128)                                                              
          

 up_sampling2d (UpSampling2D)   (None, None, None,   0           ['batch_normalization[0][0]']    
                                512)                                                              
                                                                                                  
 conv2d_3 (Conv2D)              (None, None, None,   1179904     ['up_sampling2d[0][0]']          
                                256)                                                              
                                                                                                  
 batch_normalization_1 (BatchNo  (None, None, None,   1024       ['conv2d_3[0][0]']               
 rmalization)                   256)                                                              
                                                                                                  
 up_sampling2d_1 (UpSampling2D)  (None, None, None,   0          ['batch_normalization_1[0][0]']  
          

In [6]:
Y_val_pred = [model.predict_instances(x, n_tiles=model._guess_n_tiles(x), show_tile_progress=False)[0]
              for x in tqdm(X_test)]


  0%|          | 0/14 [00:00<?, ?it/s]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 26ms/step



  7%|▋         | 1/14 [00:09<01:59,  9.20s/it]

1/1 [==============================] - 0s 24ms/step



 14%|█▍        | 2/14 [00:09<00:50,  4.25s/it]

...


 86%|████████▌ | 12/14 [00:16<00:01,  1.35it/s]

1/1 [==============================] - 0s 23ms/step



 93%|█████████▎| 13/14 [00:17<00:00,  1.36it/s]

1/1 [==============================] - 0s 24ms/step


100%|██████████| 14/14 [00:18<00:00,  1.32s/it]


In [ ]:
#plot_img_label(X_val[0],Y_val[0], lbl_title="label GT")

for i in range(14):
    plot_img_label(X_test[i],Y_val_pred[i], lbl_title="label Pred")

In [9]:
taus = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
stats = [matching_dataset(Y_test, Y_val_pred, thresh=t, show_progress=False) for t in tqdm(taus)]

100%|██████████| 9/9 [00:05<00:00,  1.52it/s]


In [10]:
stats[taus.index(0.3)]

DatasetMatching(criterion='iou', thresh=0.3, fp=1099, tp=6167, fn=530, precision=0.848747591522158, recall=0.920860086605943, accuracy=0.7910466906105695, f1=0.8833345269641194, n_true=6697, n_pred=7266, mean_true_score=0.6675361654946069, mean_matched_score=0.7249050916681341, panoptic_quality=0.6403336962425529, by_image=False)

In [12]:
# --------------------------Optimised for Speed
def get_fast_aji(true, pred):
    """AJI version distributed by MoNuSeg, has no permutation problem but suffered from 
    over-penalisation similar to DICE2.

    Fast computation requires instance IDs are in contiguous orderding i.e [1, 2, 3, 4] 
    not [2, 3, 6, 10]. Please call `remap_label` before hand and `by_size` flag has no 
    effect on the result.

    """
    true = np.copy(true)  # ? do we need this
    pred = np.copy(pred)
    true_id_list = list(np.unique(true))
    pred_id_list = list(np.unique(pred))

    true_masks = [
        None,
    ]
    for t in true_id_list[1:]:
        t_mask = np.array(true == t, np.uint8)
        true_masks.append(t_mask)

    pred_masks = [
        None,
    ]
    for p in pred_id_list[1:]:
        p_mask = np.array(pred == p, np.uint8)
        pred_masks.append(p_mask)

    # prefill with value
    pairwise_inter = np.zeros(
        [len(true_id_list) - 1, len(pred_id_list) - 1], dtype=np.float64
    )
    pairwise_union = np.zeros(
        [len(true_id_list) - 1, len(pred_id_list) - 1], dtype=np.float64
    )

    # caching pairwise
    for true_id in true_id_list[1:]:  # 0-th is background
        t_mask = true_masks[true_id]
        pred_true_overlap = pred[t_mask > 0]
        pred_true_overlap_id = np.unique(pred_true_overlap)
        pred_true_overlap_id = list(pred_true_overlap_id)
        for pred_id in pred_true_overlap_id:
            if pred_id == 0:  # ignore
                continue  # overlaping background
            p_mask = pred_masks[pred_id]
            total = (t_mask + p_mask).sum()
            inter = (t_mask * p_mask).sum()
            pairwise_inter[true_id - 1, pred_id - 1] = inter
            pairwise_union[true_id - 1, pred_id - 1] = total - inter

    pairwise_iou = pairwise_inter / (pairwise_union + 1.0e-6)
    # pair of pred that give highest iou for each true, dont care
    # about reusing pred instance multiple times
    paired_pred = np.argmax(pairwise_iou, axis=1)
    pairwise_iou = np.max(pairwise_iou, axis=1)
    # exlude those dont have intersection
    paired_true = np.nonzero(pairwise_iou > 0.0)[0]
    paired_pred = paired_pred[paired_true]
    # print(paired_true.shape, paired_pred.shape)
    overall_inter = (pairwise_inter[paired_true, paired_pred]).sum()
    overall_union = (pairwise_union[paired_true, paired_pred]).sum()

    paired_true = list(paired_true + 1)  # index to instance ID
    paired_pred = list(paired_pred + 1)
    # add all unpaired GT and Prediction into the union
    unpaired_true = np.array(
        [idx for idx in true_id_list[1:] if idx not in paired_true]
    )
    unpaired_pred = np.array(
        [idx for idx in pred_id_list[1:] if idx not in paired_pred]
    )
    for true_id in unpaired_true:
        overall_union += true_masks[true_id].sum()
    for pred_id in unpaired_pred:
        overall_union += pred_masks[pred_id].sum()

    aji_score = overall_inter / overall_union
    return aji_score



all_aji_score=[]
for m in range(14):
    aji_score = get_fast_aji(Y_test[m],Y_val_pred[m])
    all_aji_score.append(aji_score)

print(all_aji_score)
print('Total AJI: ', (sum(all_aji_score) / len(all_aji_score)))

[0.6113582577883534, 0.660976821192053, 0.6016677049887382, 0.5637696903856599, 0.5625815650828943, 0.5837558758692819, 0.6254666063982327, 0.6273184845557614, 0.6627837465058728, 0.6768129461388709, 0.653767220602629, 0.2550658594560521, 0.6919445337024495, 0.7099963027833692]
Total AJI:  0.6062332582464441
